In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from datasets import load_dataset

# Load the InsuranceQA v2 dataset
ds = load_dataset("deccan-ai/insuranceQA-v2")

print(ds)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/14.3M [00:00<?, ?B/s]

valid.jsonl:   0%|          | 0.00/2.29M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21325 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3354 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3308 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 21325
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 3354
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 3308
    })
})


In [ ]:
import pandas as pd

# Convert train set to DataFrame
train_df = pd.DataFrame(ds['train'])

# Save as CSV
train_df.to_csv('insurance_qa_train.csv', index=False)

print("CSV saved as insurance_qa_train.csv!")


CSV saved as insurance_qa_train.csv!


In [ ]:
insurance_qa = pd.read_csv('/content/insurance_qa_train.csv')
insurance_qa

,input,output
0,What Does Medicare IME Stand For?,According to the Centers for Medicare and Medi...
1,Is Long Term Care Insurance Tax Free?,"As a rule , if you buy a tax qualified long te..."
2,Can Husband Drop Wife From Health Insurance?,Can a spouse drop another spouse from health i...
3,Is Medicare Run By The Government?,Medicare Part A and Part B is provided by the ...
4,Is Medicare Run By The Government?,Definitely . It is ran by the Center for Medic...
...,...,...
21320,How Much Disability Insurance Should You Buy?,How much disbility insurance should you buy ? ...
21321,How Much Disability Insurance Should You Buy?,If you can afford it you should purchase the m...
21322,Why Do We Need Homeowners Insurance?,There isnt a legal obligation to own homeowner...
21323,Why Do We Need Homeowners Insurance?,"Homeowners insurance , and any insurance for t..."


In [ ]:
insurance_qa['input'].value_counts()

,count
input,
How much does life insurance cost per month?,15
Does life insurance pay out for suicide?,14
Can I get life insurance with high blood pressure?,13
Does life insurance get taxed?,13
Should I Take A 15 Years Term Life Insurance?,10
...,...
How Do You Notify Medicare Of A Death?,1
When Was The First Health Insurance Policy Written?,1
How Much Is Car Insurance For A Single Person?,1


In [ ]:
insurance_qa.isna().sum()

,0
input,0
output,0


In [ ]:
insurance_qa = insurance_qa.rename(columns={"input": "question", "output": "answer"})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use t5-large instead of t5-base
model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

# Update training arguments for better performance
training_args = TrainingArguments(
    output_dir="./insurance-t5-base",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    eval_strategy="no",
    save_strategy="epoch"
)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# improve preprocessing function
def preprocess(example):
    input_text = f"insurance query: {example['question'].strip()}"
    target_text = example["answer"].strip()

    # Tokenize input and output
    model_inputs = tokenizer(
        input_text,
        max_length=128,
        truncation=True,
        padding="max_length"  # Ensures consistent tensor sizes
    )

    labels = tokenizer(
        target_text,
        max_length=128,
        truncation=True,
        padding="max_length"
    )["input_ids"]

    # Replace padding tokens with -100 to ignore in loss calculation
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]

    model_inputs["labels"] = labels
    return model_inputs


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(insurance_qa)

In [ ]:
dataset = dataset.map(preprocess, remove_columns=["question", "answer"])

Map:   0%|          | 0/21325 [00:00<?, ? examples/s]

In [ ]:
# Train-Validation Split

dataset = dataset.train_test_split(test_size=0.1)  # 90% training, 10% validation
train_dataset = dataset["train"]
val_dataset = dataset["test"]

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# Load Model & Training Setup
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Load pre-trained T5 model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Data collator handles padding during training
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./insurance-t5-base",
    run_name="insurance-chatbot-training",  # Better to reflect model
    per_device_train_batch_size=8,      # Reduce if memory issues with t5-large
    per_device_eval_batch_size=8,
    num_train_epochs=3,                 # More training time helps generalization
    learning_rate=3e-5,                 # Lower LR for large models is common
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    fp16 = False,
    load_best_model_at_end=True,
    metric_for_best_model="loss",       # Choose best model by loss
    greater_is_better=False             # Lower loss = better
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Initialize Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-25-c193f928227a>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#  Train the Model
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.797700,2.610325
2,2.760000,2.558654


Epoch,Training Loss,Validation Loss
1,2.797700,2.610325
2,2.760000,2.558654
3,2.649700,2.544830


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=7197, training_loss=2.7729933050849196, metrics={'train_runtime': 4054.5541, 'train_samples_per_second': 14.2, 'train_steps_per_second': 1.775, 'total_flos': 8765339876720640.0, 'train_loss': 2.7729933050849196, 'epoch': 3.0})

In [ ]:
# ✅ Step 11: Save Model & Tokenizer
model.save_pretrained("/content/insurance_t5_model")
tokenizer.save_pretrained("/content/insurance_t5_model")

('/content/insurance_t5_model/tokenizer_config.json',
 '/content/insurance_t5_model/special_tokens_map.json',
 '/content/insurance_t5_model/spiece.model',
 '/content/insurance_t5_model/added_tokens.json',
 '/content/insurance_t5_model/tokenizer.json')

In [ ]:
!zip -r /content/insurance_t5_model.zip /content/insurance_t5_model

# Finally, download it
from google.colab import files
files.download("/content/insurance_t5_model.zip")


  adding: content/insurance_t5_model/ (stored 0%)
  adding: content/insurance_t5_model/spiece.model (deflated 48%)
  adding: content/insurance_t5_model/special_tokens_map.json (deflated 86%)
  adding: content/insurance_t5_model/tokenizer_config.json (deflated 95%)
  adding: content/insurance_t5_model/config.json (deflated 63%)
  adding: content/insurance_t5_model/tokenizer.json (deflated 74%)
  adding: content/insurance_t5_model/model.safetensors (deflated 8%)
  adding: content/insurance_t5_model/generation_config.json (deflated 29%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive

# Mount drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# !cp -r /content/insurance_t5_model /content/drive/MyDrive/

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "/content/drive/MyDrive/insurance_t5_model"

tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)


In [ ]:
# Initialize an empty history
# conversation_history = ""

def clean_bot_response(response):
    return response.replace('-LRB-', '').replace('-RRB-', '').replace('?','.')


def chat_with_bot(user_input):

    # Encode
    input_ids = tokenizer.encode(user_input, return_tensors="pt", truncation=True, max_length=512)

    # Generate
    outputs = model.generate(
        input_ids,
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # Decode
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

    reply = clean_bot_response(reply)

    return reply





In [ ]:
print("Welcome to the Smart Insurance Bot! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break

    bot_reply = chat_with_bot(user_input)
    print(f"Bot: {bot_reply}\n")


Welcome to the Smart Insurance Bot! Type 'exit' to stop.

You: how can i claim an insurance?
Bot: How can I claim an insurance policy . The first step is to contact your insurance company to find out if you have a claim . If you do n't have one , you can contact the company directly to have them go over the policy with you  e.g. :  i 'm not able to help you with this !

You: what is a rentes insurance?
Bot: A renters insurance policy is a policy that covers the loss of your personal property if you are unable to pay your rent . The policy will cover the damage to your property that is caused by an accident , theft or other loss that you may have caused in the past  e.g. your car 's vandalism ) and will protect you from any claims that may be made against you by the insurance company unless you have been able to provide proof of insurance for at least 10 years and the rent you owes

You: what are the types of insurances?
Bot: What are the types of insurance policies that are available t